# データ前処理

In [1]:
import pandas as pd
from io import StringIO

data = """A,B,C,D
1.0,2.0,3.0,
10.0,20.0,,30.0"""
df = pd.read_csv(StringIO(data))
print(df)

      A     B    C     D
0   1.0   2.0  3.0   NaN
1  10.0  20.0  NaN  30.0


In [2]:
print(df.isnull().sum())

A    0
B    0
C    1
D    1
dtype: int64


In [7]:
df

,A,B,C,D
0,1.0,2.0,3.0,NaN
1,10.0,20.0,NaN,30.0


In [10]:
df.dropna(axis=1)

,A,B
0,1.0,2.0
1,10.0,20.0


In [9]:
df.dropna(axis=0)

,A,B,C,D


In [11]:
df.dropna(how="all")

,A,B,C,D
0,1.0,2.0,3.0,NaN
1,10.0,20.0,NaN,30.0


In [12]:
df.dropna(thresh=5)

,A,B,C,D


In [13]:
df.dropna(subset=["A"])

,A,B,C,D
0,1.0,2.0,3.0,NaN
1,10.0,20.0,NaN,30.0


## 補完法

In [16]:
from sklearn.impute import SimpleImputer
# 平均値で欠損値を補完するためのインスタンスを作成
imp = SimpleImputer(strategy = 'mean')
# 欠損値を補完
imp.fit(df)
imp.transform(df)

# = 以前の書き方 = #
# Imputerクラスは0.20で廃止予定となっており、0.22で削除されます。そのため今後はSimpleInputerクラスを使用してください。

# from sklearn.preprocessing import Imputer
# 欠測値補完のインスタンスを生成(平均値補完)
# imr = Imputer(missing_values="NaN", strategy="mean", axis=0)
# データを適合
# imr = imr.fit(df) # fit()の内部処理 https://github.com/scikit-learn/scikit-learn/blob/a24c8b46/sklearn/preprocessing/imputation.py#L125
# 補完を実行
# imputed_data = imr.transform(df.values)
# print(imputed_data)

# """
# 補足で、2点注意が必要です。
# 1.Imputerクラスを使う場合文字列が含まれていると使用できないので注意が必要です。
# 2. また、fit()で渡せるデータフレームは、全体の値が数値もしくは小数点型である必要があります。
# 文字列などが含まれている全体のデータを使うと『stringからfloatにconvertできません』と言ったエラーが出てしまいます。
# ですので、convertエラーが出ないようにするためには、特定の数値の部分のカラムのみを取り出してfit()に渡す必要があります。

# """

array([[ 1.,  2.,  3., 30.],
       [10., 20.,  3., 30.]])

In [17]:
import pandas as pd
df = pd.DataFrame()

In [21]:
import pandas as pd
df = pd.read_csv('./train2.csv')
titanic_y = df['Survived']
titanic_x = df.drop(['Survived'],axis=1)
titanic_x  = titanic_x[["Pclass","Age","Sex"]]
titanic_x.head() # 先頭5件取得

,Pclass,Age,Sex
0,3,22.0,male
1,1,38.0,female
2,3,26.0,female
3,1,35.0,female
4,3,35.0,male


In [26]:
import pandas as pd
import numpy as np
df = pd.DataFrame()

In [24]:
from sklearn.preprocessing import LabelEncoder
enc = LabelEncoder()
label_encoder = enc.fit(titanic_x[["Sex"]])
print('Categorical classes:',label_encoder.classes_)

integer_classes = label_encoder.transform(label_encoder.classes_)
print('Integer classes:',integer_classes)
t = label_encoder.transform(titanic_x[["Sex"]])
titanic_x["Sex"] = t

print(titanic_x[["Sex"]])


Categorical classes: [0 1]
Integer classes: [0 1]
     Sex
0      1
1      0
2      0
3      0
4      1
..   ...
886    1
887    0
888    0
889    1
890    1

[891 rows x 1 columns]


In [27]:
from sklearn.preprocessing import OneHotEncoder
enc = LabelEncoder()
label_encoder = enc.fit(titanic_x ["Pclass"])
print("Categorical classes:", label_encoder.classes_)
integer_classes = label_encoder.transform(label_encoder.classes_).reshape(3, 1)
print("Integer classes:", integer_classes)
enc = OneHotEncoder()
one_hot_encoder = enc.fit(integer_classes)

#最初に、Label Encoderを使ってpclassを0-2に直す
num_of_rows = titanic_x.shape[0]
#transform()に渡すためには、[n_samples, n_features]の行列を渡す必要があるため (1313, 1)にreshape
t = label_encoder.transform(titanic_x["Pclass"]).reshape(num_of_rows, 1)
#次に、OneHotEncoderを使ってデータを1, 0に変換
new_features = one_hot_encoder.transform(t)
#1,0になおしてデータを統合する
titanic_x = np.concatenate([titanic_x, new_features.toarray()], axis = 1)
#OnehotEncoderをする前のpclassのデータを削除する
titanic_x = np.delete(titanic_x, [0], 1)
#特徴量の名前を更新する
feature_names = ['age', 'sex', 'first class', 'second class', 'third class']

print(feature_names)
print(titanic_x[0],titanic_y[0])

Categorical classes: [1 2 3]
Integer classes: [[0]
 [1]
 [2]]
['age', 'sex', 'first class', 'second class', 'third class']
[22.  1.  0.  0.  1.] 0


## 特徴量のスケーリング
機械学習の前処理として、特徴量のスケーリングがあります。
スケーリングの例としては、「平均を0，分散を1とするZスコア(標準化)」、「最小値を0，最大値を1とする0-1スケーリング(正規化)」などがあげられます。

## 標準化

In [28]:
import numpy as np
def zscore(x, axis = None):
    x_mean = x.mean(axis=axis, keepdims=True)
    x_std  = np.std(x, axis=axis, keepdims=True)
    z_score = (x-x_mean)/x_std
    return z_score

a = np.random.randint(10, size=(2,5))
print(a) # 実行結果は毎回異なります。

b = zscore(a)
print(b)
"""
array([[-1.41421356,  0.        ,  1.41421356,  0.35355339,  1.41421356],
       [-1.41421356,  0.35355339, -1.06066017,  0.70710678, -0.35355339]])
"""

# sklearnを使った場合
from sklearn.preprocessing import StandardScaler
# 標準化
sc = StandardScaler()
# 引数にdataという変数を入れた場合です
# X_std = sc.fit_transform(data)

[[8 4 5 0 5]
 [0 1 4 0 0]]
[[ 1.94700365  0.47756693  0.84492611 -0.99186978  0.84492611]
 [-0.99186978 -0.6245106   0.47756693 -0.99186978 -0.99186978]]


## 正規化

In [29]:
import numpy as np
def min_max(x, axis=None):
    x_min = x.min(axis=axis, keepdims=True)
    x_max = x.max(axis=axis, keepdims=True)
    result = (x-x_min)/(x_max-x_min)
    return result

a = np.random.randint(10, size=(2,5))
print(a)
print(zscore)


# sklearnを使った場合
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler([-1,1])
# 引数にdataという変数を入れた場合です
# X_MinMaxScaler = scaler.fit_transform(data)

[[9 9 6 6 1]
 [4 9 2 0 2]]
<function zscore at 0x1a25331f80>


標準化と正規化の使い分け  
標準化と正規化の特徴を確認しましょう。  
標準化では、データを平均0、標準偏差が1になるように変換する正規化法でしたが、一般的に  標準化を用いる場合は、最大値及び最小値が決まっていない場合や外れ値が存在する場合に  利用します。  
基本的には正規化ではなく、標準化を利用する事が多いです。  
正規化の場合は、外れ値が大きく影響してしまうためです。  
一方、正規化では主に0、1の範囲内にスケーリングしますが、最大値及び最小値が決まっている場合などに利用します。  
画像処理などでは学習コストを下げる事ができるため、良く用いられます。